# PPIxFAC (Plasmapause boundary)

> Abstract: Access to the `PPIxFAC` product from the PRISM project. This gives information about the boundaries of small-scale field-aligned currents (SSFAC) and the associated midnight plasmapause (PP) index.

See also:
- https://swarmhandbook.earth.esa.int/catalogue/SW_PPIxFAC_2F
- MIT notebook: https://notebooks.vires.services/notebooks/03j1_demo-prism-mitx
- More notebooks: https://nbviewer.org/github/pacesm/jupyter_notebooks/tree/master/PRISM/
- Project page and documentation: https://earth.esa.int/eogateway/activities/plasmapause-related-boundaries-in-the-topside-ionosphere-as-derived-from-swarm-measurements

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
from viresclient import SwarmRequest

## Collection information

The dataset is split over a default collection, and a subcollection:
- `SW_OPER_PPIxFAC_2F` contains the plasmapause index (PPI)
- `SW_OPER_PPIxFAC_2F:ID` identifies boundaries of SSSFAC (see `PointType`)

Collections available:

| Collection | Dataset |
|---|---|
|`SW_OPER_PPIAFAC_2F`| Swarm A PPI and SSFAC boundaries |
|`SW_OPER_PPIAFAC_2F:ID`| Swarm A edges of the SSFAC boundaries ("ID" dataset) |
|`SW_OPER_PPIBFAC_2F`| Swarm B PPI and SSFAC boundaries |
|`SW_OPER_PPIBFAC_2F:ID`| Swarm B edges of the SSFAC boundaries ("ID" dataset) |
|`SW_OPER_PPICFAC_2F`| Swarm C PPI and SSFAC boundaries |
|`SW_OPER_PPICFAC_2F:ID`| Swarm C edges of the SSFAC boundaries ("ID" dataset) |

Available "measurements" in each collection type:

In [ ]:
request = SwarmRequest()
print("PPIxFAC variables:\n", request.available_measurements("SW_OPER_PPIAFAC_2F"))
print("PPIxFAC:ID variables:\n", request.available_measurements("SW_OPER_PPIAFAC_2F:ID"))

$$R_E$$

### `SW_OPER_PPIxFAC_2F`

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Counter` | `Counter` | $$-$$ | vector[2] | Swarm orbit and quarter orbit counters |
| `Timestamp` | `Timestamp` | $$-$$ | scalar | UTC time of the SSFAC boundary |
| `Latitude` | `Latitude` | $$\text{deg}$$ | scalar | ITRF geocentric latitude of the SSFAC boundary |
| `Longitude` | `Longitude` | $$\text{deg}$$ | scalar | ITRF geocentric longitude of the SSFAC boundary |
| `Radius` | `Radius` | $$\text{m}$$ | scalar | ITRF geocentric radius of the SSFAC boundary |
| `Latitude_QD` | `Latitude_QD` | $$\text{deg}$$ | scalar | Quasi-dipole latitude of the SSFAC boundary |
| `Longitude_QD` | `Longitude_QD` | $$\text{deg}$$ | scalar | Quasi-dipole longitude of the SSFAC boundary |
| `MLT_QD` | `MLT` | $$\text{hour}$$ | scalar | Magnetic local time (QD) of the SSFAC boundary |
| `L_value` | `L_value` | $$-$$ | scalar | L-value in earth radii of the SSFAC boundary |
| `SZA` | `SZA` | $$\text{deg}$$ | scalar | Solar zenith angle of the SSFAC boundary |
| `Sigma` | `Sigma` | $$-$$ | scalar | Standard deviation of the linear fit of S at the boundary |
| `PPI` | `PPI` | $$R_E$$ | scalar | midnight plasmapause index |
| `dL` | `dL` | $$\text{deg}$$ | scalar |  boundary width in Earth radii |
| `Quality` | `Quality` | $$-$$ | vector [7] |  Quality indicators of SSFAC boundary detection |

The `Quality` vector flags contain integer values from -1 to 3 indicating the quality of the MIT as a whole.
Positive values indicate good or acceptable observation, while -1 means a likely false observation.
For all flags, a 0 value means that the information needed to derive the flag is not available.
In general, greater numbers indicate higher quality. For more details, see the product documentation.

### `SW_OPER_PPIxFAC_2F:ID`

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Counter` | `Counter` | $$-$$ | vector[2] | Swarm orbit and quarter orbit counters |
| `Timestamp` | `Timestamp_ID` | $$-$$ | scalar | UTC time of the observation |
| `Latitude` | `Latitude_ID` | $$\text{deg}$$ | scalar | ITRF geocentric latitude |
| `Longitude` | `Longitude_ID` | $$\text{deg}$$ | scalar | ITRF geocentric longitude |
| `Radius` | `Radius_ID` | $$\text{m}$$ | scalar | ITRF geocentric radius |
| `Latitude_QD` | `Latitude_QD_ID` | $$\text{deg}$$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | `Longitude_QD_ID` | $$\text{deg}$$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | `MLT_ID` | $$\text{hour}$$ | scalar | Magnetic local time (QD) |
| `L_value` | `L_value_ID` | $$-$$ | scalar | L-value in earth radii |
| `SZA` | `SZA_ID` | $$\text{deg}$$ | scalar | Solar zenith angle |
| `Position_Quality` | `Position_Quality_ID` | $$-$$ | scalar | detection quality of each single position |
| `PointType` | n/a | $$-$$ | scalar | point type derived from the source `*_ID` column |

`Position_Quality` describes the overall quality of the corresponding boundary/position. The possible values and their meaning are 3: highest quality; 2: fair quality;
1: uncertain detection; -1: a likely false positive detection; -2: the corresponding boundary/position is not defined. For more details, see the product documentation.

These are the possible `PointType` values for the PPIxFAC_2F products:

| Value | Hex. | Bin. | ID Column | Meaning |
|---|---|---|---|---|
| 0 | 0x0 | 0b0000 | 1 | equatorward edge of SSFAC boundary |
| 1 | 0x1 | 0b0001 | 2 | poleward edge of SSFAC boundary |

## PPIxFAC example

In [ ]:
start = dt.datetime(2016, 1, 1)
end = dt.datetime(2016, 1, 8)

In [ ]:
def attach_missing_units(data, ds):
    """TO FIX"""
    with data.contents[0].open_cdf() as cdf:
        for data_var in ds.data_vars:
            units = cdf.varattsget(data_var).get("UNIT", "-")
            ds[data_var].attrs["units"] = units
    return ds

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_PPIAFAC_2F")
request.set_products(
    measurements=['Counter', 'Latitude_QD', 'Longitude_QD', 'MLT_QD', 'L_value', 'SZA', 'Sigma', 'PPI', 'dL', 'Quality'],
    auxiliaries=["QDLat", "QDLon", "MLT", "OrbitNumber"],
)
data = request.get_between(
    start_time=start, end_time=end, asynchronous=False, show_progress=False
)
ds = data.as_xarray()
ds = attach_missing_units(data, ds)
ds

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_PPIAFAC_2F:ID")
request.set_products(
    measurements=['Counter', 'Latitude_QD', 'Longitude_QD', 'MLT_QD', 'L_value', 'SZA', 'Position_Quality', 'PointType'],
    auxiliaries=["QDLat", "QDLon", "MLT", "OrbitNumber"],
)
data = request.get_between(
    start_time=start, end_time=end, asynchronous=False, show_progress=False
)
ds_id = data.as_xarray()
ds_id = attach_missing_units(data, ds_id)
ds_id

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5), sharex=True)
_equatorward = ds_id.where(ds_id["PointType"] == 0, drop=True)
_poleward = ds_id.where(ds_id["PointType"] == 1, drop=True)
axes[0].plot(_equatorward["Timestamp"], _equatorward["QDLat"], color="grey", linestyle="--")
axes[0].plot(_poleward["Timestamp"], _poleward["QDLat"], color="grey", linestyle="--")
axes[0].plot(ds["Timestamp"], ds["QDLat"], color="black", linestyle="-")
axes[1].plot(ds["Timestamp"], ds["PPI"])
axes[0].set_ylim(40, 90)
axes[0].set_title("Boundaries of SSFAC")
axes[0].set_ylabel("QDLat")
axes[1].set_title("Plasmapause index")
axes[1].set_ylabel("PPI [$R_E$]")
axes[0].grid()
axes[1].grid()